<a href="https://colab.research.google.com/github/Arminsbss/Reinforcement-learning-N-N-grid/blob/main/reinforcement_learning_n_n_grid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

Grid = pd.read_excel('Grid.xlsx',header=None)
Grid = Grid.values

In [ ]:
Start_State_indexes=[29,0]
End_State_indexes=[0,29]
Grid[Start_State_indexes[0]][Start_State_indexes[1]]

def Right_Action(get_cell,row,column):
  return get_cell[row,column+1]

def Up_Action(get_cell,row,column):
  return get_cell[row-1,column]

Place=Grid[Start_State_indexes[0]][Start_State_indexes[1]]
Place=Right_Action(Grid,Start_State_indexes[0],Start_State_indexes[1])
print(Place)

8
